In [ ]:
!python --version

In [ ]:
!pip list

In [ ]:
!pip install --disable-pip-version-check -q pip --upgrade > /dev/null
!pip install --disable-pip-version-check -q wrapt --upgrade > /dev/null

In [ ]:
!pip install --disable-pip-version-check -q awscli==1.18.216 boto3==1.16.56 botocore==1.19.56

In [ ]:
!pip install --disable-pip-version-check -q sagemaker==2.29.0
!pip install --disable-pip-version-check -q smdebug==1.0.1
!pip install --disable-pip-version-check -q sagemaker-experiments==0.1.26

In [ ]:

!conda install -y pytorch==1.6.0 -c pytorch

In [ ]:
!pip install imblearn

In [ ]:
!pip install time

In [ ]:
!pip install --disable-pip-version-check -q tensorflow==2.8.1

In [ ]:
!pip install --disable-pip-version-check -q transformers==3.5.1

In [ ]:
!pip install --disable-pip-version-check -q torchserve==0.3.0
!pip install --disable-pip-version-check -q torch-model-archiver==0.3.0

In [ ]:
!pip install --disable-pip-version-check -q PyAthena==2.1.0

In [ ]:

!pip install --disable-pip-version-check -q SQLAlchemy==1.3.22

In [ ]:
!pip install --disable-pip-version-check -q awswrangler==2.15.0

In [ ]:
!pip install --disable-pip-version-check -q stepfunctions==2.0.0rc1

In [ ]:
!conda install -y zip

In [ ]:
!pip install --disable-pip-version-check -q matplotlib==3.1.3

In [ ]:
!pip install --disable-pip-version-check -q seaborn==0.10.0

In [ ]:
!pip install protobuf==3.20.*
!python --version

In [ ]:
!pip list

In [ ]:
setup_dependencies_passed = True

In [ ]:
%store setup_dependencies_passed

In [ ]:
%store

In [ ]:
!pip install --disable-pip-version-check -q sagemaker==2.29.0
import boto3
import sagemaker

session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

s3 = boto3.Session().client(service_name="s3", region_name=region)

In [ ]:

setup_s3_bucket_passed = False

In [ ]:
print("Default bucket: {}".format(bucket))

In [ ]:
%%bash

aws s3 ls s3://${bucket}/

In [ ]:
from botocore.client import ClientError

response = None

try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print("[ERROR] Cannot find bucket {} in {} due to {}.".format(bucket, response, e))

In [ ]:
%store setup_s3_bucket_passed

In [ ]:
%store

In [ ]:
%store

In [ ]:
import boto3
import sagemaker
import time
from time import gmtime, strftime

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

from botocore.config import Config

config = Config(retries={"max_attempts": 10, "mode": "adaptive"})

iam = boto3.client("iam", config=config)

In [ ]:
role_name = role.split("/")[-1]

print("Role name: {}".format(role_name))

In [ ]:
setup_iam_roles_passed = False

In [ ]:
admin = False
post_policies = iam.list_attached_role_policies(RoleName=role_name)["AttachedPolicies"]
for post_policy in post_policies:
    if post_policy["PolicyName"] == "AdministratorAccess":
        admin = True
        break

setup_iam_roles_passed = True
print("[OK] You are all set up to continue with this workshop!")

In [ ]:
if not admin:
    pre_policies = iam.list_attached_role_policies(RoleName=role_name)["AttachedPolicies"]

    required_policies = ["IAMFullAccess"]

    for pre_policy in pre_policies:
        for role_req in required_policies:
            if pre_policy["PolicyName"] == role_req:
                print("Attached: {}".format(pre_policy["PolicyName"]))
                try:
                    required_policies.remove(pre_policy["PolicyName"])
                except:
                    pass

    if len(required_policies) > 0:
        print(
            "*************** [ERROR] You need to attach the following policies in order to continue with this workshop *****************\n"
        )
        for required_policy in required_policies:
            print("Not Attached: {}".format(required_policy))
    else:
        print("[OK] You are all set to continue with this notebook!")
else:
    print("[OK] You are all set to continue with this notebook!")

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "AdministratorAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "AmazonSageMakerFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "IAMFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "AmazonS3FullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "ComprehendFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "AmazonAthenaFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "SecretsManagerReadWrite"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "AmazonRedshiftFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "AmazonEC2ContainerRegistryFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "AWSStepFunctionsFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "AmazonKinesisFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "AmazonKinesisFirehoseFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "AmazonKinesisAnalyticsFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
post_policies = iam.list_attached_role_policies(RoleName=role_name)["AttachedPolicies"]

required_policies = [
    "AdministratorAccess",
    "SecretsManagerReadWrite",
    "IAMFullAccess",
    "AmazonS3FullAccess",
    "AmazonAthenaFullAccess",
    "ComprehendFullAccess",
    "AmazonEC2ContainerRegistryFullAccess",
    "AmazonRedshiftFullAccess",
    "AWSStepFunctionsFullAccess",
    "AmazonSageMakerFullAccess",
    "AmazonKinesisFullAccess",
    "AmazonKinesisFirehoseFullAccess",
    "AmazonKinesisAnalyticsFullAccess",
]

admin = False

for post_policy in post_policies:
    if post_policy["PolicyName"] == "AdministratorAccess":
        admin = True
        try:
            required_policies.remove(post_policy["PolicyName"])
        except:
            break
    else:
        try:
            required_policies.remove(post_policy["PolicyName"])
        except:
            pass

if not admin and len(required_policies) > 0:
    print("*************** [ERROR] RE-RUN THIS NOTEBOOK *****************")
    for required_policy in required_policies:
        print("Not Attached: {}".format(required_policy))
else:
    setup_iam_roles_passed = True
    print("[OK] You are all set up to continue with this workshop!")

In [ ]:
%store setup_iam_roles_passed

In [ ]:
%store

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}